In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict


In [2]:

def analyze_and_deduplicate_customers(file_path):
    """
    Analyze duplicate Customer IDs and remove duplicates based on priority rules.
    
    Args:
        file_path (str): Path to the CSV file
    
    Returns:
        tuple: (deduplicated_dataframe, analysis_results)
    """
    
    # Load the data
    df = pd.read_csv(file_path)
    print(f"Original dataset: {len(df)} records")
    print(f"Unique Customer IDs: {df['Customer ID'].nunique()}")
    print(f"Duplicate Customer ID records: {len(df) - df['Customer ID'].nunique()}")
    
    # Identify duplicates
    duplicate_ids = df[df.duplicated(subset=['Customer ID'], keep=False)]['Customer ID'].unique()
    print(f"Number of Customer IDs with duplicates: {len(duplicate_ids)}")
    
    # Analysis of changing columns for duplicate Customer IDs
    print("\n" + "="*60)
    print("ANALYSIS: Which columns change for duplicate Customer IDs")
    print("="*60)
    
    column_changes = defaultdict(int)  # Count of records where each column changes
    changing_columns_per_id = []  # Track which columns change for each duplicate ID
    
    for customer_id in duplicate_ids:
        customer_records = df[df['Customer ID'] == customer_id].copy()
        
        # Check which columns have different values for this customer
        changing_cols = []
        for col in df.columns:
            if col != 'Customer ID':  # Skip Customer ID itself
                unique_values = customer_records[col].nunique()
                if unique_values > 1:
                    changing_cols.append(col)
                    column_changes[col] += len(customer_records)  # Count all records for this ID
        
        changing_columns_per_id.append({
            'Customer ID': customer_id,
            'num_records': len(customer_records),
            'changing_columns': changing_cols
        })
    
    # Display column change analysis
    print(f"\nColumns that change across duplicate Customer IDs:")
    print(f"{'Column Name':<20} {'Records Affected':<20} {'Percentage':<15}")
    print("-" * 55)
    
    total_duplicate_records = sum(len(df[df['Customer ID'] == cid]) for cid in duplicate_ids)
    
    for col, count in sorted(column_changes.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total_duplicate_records) * 100
        print(f"{col:<20} {count:<20} {percentage:.1f}%")
    
    # Show examples of Customer IDs and their changing columns
    print(f"\nExamples of Customer IDs with changing columns:")
    print(f"{'Customer ID':<15} {'Records':<10} {'Changing Columns'}")
    print("-" * 70)
    
    for item in changing_columns_per_id[:10]:  # Show first 10 examples
        changing_cols_str = ', '.join(item['changing_columns']) if item['changing_columns'] else 'None'
        print(f"{item['Customer ID']:<15} {item['num_records']:<10} {changing_cols_str}")
    
    # Deduplication process
    print("\n" + "="*60)
    print("DEDUPLICATION PROCESS")
    print("="*60)
    
    # Create a copy for deduplication
    df_dedup = df.copy()
    
    # For each duplicate Customer ID, keep the record with:
    # 1. Higher Age (primary criterion)
    # 2. Higher Work_Experience (secondary criterion if ages are equal)
    
    def select_best_record(group):
        """Select the best record from a group of duplicates."""
        if len(group) == 1:
            return group
        
        # Sort by Age (descending), then by Work_Experience (descending)
        # Handle NaN values by treating them as lowest priority
        group_sorted = group.sort_values(
            ['Age', 'Work_Experience'], 
            ascending=[False, False],
            na_position='last'
        )
        
        return group_sorted.iloc[[0]]  # Return first row as DataFrame
    
    # Apply deduplication
    df_deduplicated = df_dedup.groupby('Customer ID').apply(select_best_record).reset_index(drop=True)
    
    print(f"After deduplication: {len(df_deduplicated)} records")
    print(f"Records removed: {len(df) - len(df_deduplicated)}")
    print(f"Unique Customer IDs: {df_deduplicated['Customer ID'].nunique()}")
    
    # Show some examples of deduplication decisions
    print(f"\nExamples of deduplication decisions:")
    print(f"{'Customer ID':<15} {'Original Records':<18} {'Selected Record Criteria'}")
    print("-" * 80)
    
    for customer_id in list(duplicate_ids)[:5]:  # Show first 5 examples
        original_records = df[df['Customer ID'] == customer_id].copy()
        selected_record = df_deduplicated[df_deduplicated['Customer ID'] == customer_id].iloc[0]
        
        # Show the decision criteria
        ages = original_records['Age'].tolist()
        work_exps = original_records['Work_Experience'].tolist()
        selected_age = selected_record['Age']
        selected_work_exp = selected_record['Work_Experience']
        
        decision_reason = f"Age: {selected_age} (from {ages}), Work_Exp: {selected_work_exp} (from {work_exps})"
        print(f"{customer_id:<15} {len(original_records):<18} {decision_reason}")
    
    # Summary statistics
    print(f"\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)
    print(f"Original records: {len(df)}")
    print(f"Deduplicated records: {len(df_deduplicated)}")
    print(f"Records removed: {len(df) - len(df_deduplicated)}")
    print(f"Duplicate Customer IDs processed: {len(duplicate_ids)}")
    print(f"Most frequently changing column: {max(column_changes.items(), key=lambda x: x[1])[0] if column_changes else 'None'}")
    
    return df_deduplicated, {
        'column_changes': dict(column_changes),
        'duplicate_ids_count': len(duplicate_ids),
        'records_removed': len(df) - len(df_deduplicated),
        'changing_columns_per_id': changing_columns_per_id
    }

# Additional function to save the deduplicated data
def save_deduplicated_data(df_deduplicated, output_path='customer_segmentation_deduplicated.csv'):
    """Save the deduplicated dataframe to a CSV file."""
    df_deduplicated.to_csv(output_path, index=False)
    print(f"\nDeduplicated data saved to: {output_path}")

# Function to show detailed comparison for specific Customer IDs
def show_detailed_comparison(df, customer_ids_to_show=None, num_examples=3):
    """Show detailed before/after comparison for specific Customer IDs."""
    
    if customer_ids_to_show is None:
        # Get some duplicate IDs for demonstration
        duplicate_ids = df[df.duplicated(subset=['Customer ID'], keep=False)]['Customer ID'].unique()
        customer_ids_to_show = duplicate_ids[:num_examples]
    
    print(f"\n" + "="*80)
    print("DETAILED COMPARISON FOR DUPLICATE CUSTOMER IDs")
    print("="*80)
    
    for customer_id in customer_ids_to_show:
        records = df[df['Customer ID'] == customer_id].copy()
        if len(records) > 1:
            print(f"\nCustomer ID: {customer_id}")
            print("-" * 40)
            
            for idx, (_, record) in enumerate(records.iterrows()):
                print(f"Record {idx + 1}:")
                for col in records.columns:
                    if col != 'Customer ID':
                        print(f"  {col}: {record[col]}")
                print()

# Example usage:
if __name__ == "__main__":
    # Replace 'Customer Segmentation.csv' with your actual file path
    file_path = 'Customer Segmentation.csv'
    
    try:
        # Run the analysis and deduplication
        df_deduplicated, analysis_results = analyze_and_deduplicate_customers(file_path)
        
        # Save the deduplicated data
        save_deduplicated_data(df_deduplicated)
        
        # Show detailed comparison for a few examples
        original_df = pd.read_csv(file_path)
        show_detailed_comparison(original_df, num_examples=2)
        
        print(f"\n✅ Process completed successfully!")
        print(f"📊 Analysis results available in 'analysis_results' variable")
        print(f"🔄 Deduplicated data available in 'df_deduplicated' variable")
        
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        print("Please make sure the CSV file exists in the specified path.")
    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

❌ File not found: Customer Segmentation.csv
Please make sure the CSV file exists in the specified path.


In [3]:
datapath = "/Users/whysocurious/Documents/MLDSAIProjects/cust-seg-case-study/data"
dfcust = pd.read_csv(datapath+'/Customer Segmentation.csv')
dfcust.head()

,Customer ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size
0,460394,Female,No,32,Yes,Healthcare,9.0,Low,3.0
1,461798,Female,No,56,Yes,Artist,0.0,Low,1.0
2,464976,Male,Yes,75,No,Lawyer,1.0,Low,1.0
3,462104,Male,Yes,39,No,Entertainment,9.0,Low,2.0
4,464829,Male,No,22,No,Healthcare,9.0,Low,5.0


In [4]:
df= dfcust.copy()
print (df.shape)

(10695, 9)


In [5]:
print(f"Original dataset: {len(df)} records")
print(f"Unique Customer IDs: {df['Customer ID'].nunique()}")
print(f"Duplicate Customer ID records: {len(df) - df['Customer ID'].nunique()}")

# Identify duplicates
duplicate_ids = df[df.duplicated(subset=['Customer ID'], keep=False)]['Customer ID'].unique()
print(f"Number of Customer IDs with duplicates: {len(duplicate_ids)}")


Original dataset: 10695 records
Unique Customer IDs: 8363
Duplicate Customer ID records: 2332
Number of Customer IDs with duplicates: 2332


In [6]:

# Analysis of changing columns for duplicate Customer IDs
print("\n" + "="*60)
print("ANALYSIS: Which columns change for duplicate Customer IDs")
print("="*60)

column_changes = defaultdict(int)  # Count of records where each column changes
changing_columns_per_id = []  # Track which columns change for each duplicate ID

for customer_id in duplicate_ids:
    customer_records = df[df['Customer ID'] == customer_id].copy()
    
    # Check which columns have different values for this customer
    changing_cols = []
    for col in df.columns:
        if col != 'Customer ID':  # Skip Customer ID itself
            unique_values = customer_records[col].nunique()
            if unique_values > 1:
                changing_cols.append(col)
                column_changes[col] += len(customer_records)  # Count all records for this ID
    
    changing_columns_per_id.append({
        'Customer ID': customer_id,
        'num_records': len(customer_records),
        'changing_columns': changing_cols
    })

# Display column change analysis
print(f"\nColumns that change across duplicate Customer IDs:")
print(f"{'Column Name':<20} {'Records Affected':<20} {'Percentage':<15}")
print("-" * 55)

total_duplicate_records = sum(len(df[df['Customer ID'] == cid]) for cid in duplicate_ids)

for col, count in sorted(column_changes.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / total_duplicate_records) * 100
    print(f"{col:<20} {count:<20} {percentage:.1f}%")

# Show examples of Customer IDs and their changing columns
print(f"\nExamples of Customer IDs with changing columns:")
print(f"{'Customer ID':<15} {'Records':<10} {'Changing Columns'}")
print("-" * 70)

for item in changing_columns_per_id[:10]:  # Show first 10 examples
    changing_cols_str = ', '.join(item['changing_columns']) if item['changing_columns'] else 'None'
    print(f"{item['Customer ID']:<15} {item['num_records']:<10} {changing_cols_str}")



ANALYSIS: Which columns change for duplicate Customer IDs

Columns that change across duplicate Customer IDs:
Column Name          Records Affected     Percentage     
-------------------------------------------------------
Age                  4118                 88.3%
Work_Experience      2284                 49.0%

Examples of Customer IDs with changing columns:
Customer ID     Records    Changing Columns
----------------------------------------------------------------------
460394          2          None
460962          2          Age
460333          2          Age, Work_Experience
467208          2          Age
466301          2          Age, Work_Experience
461363          2          Age
461522          2          Age
462788          2          Age
462971          2          Age
461882          2          Age


In [7]:
# Deduplication process
print("\n" + "="*60)
print("DEDUPLICATION PROCESS")
print("="*60)

# Create a copy for deduplication
df_dedup = df.copy()

# For each duplicate Customer ID, keep the record with:
# 1. Higher Age (primary criterion)
# 2. Higher Work_Experience (secondary criterion if ages are equal)

def select_best_record(group):
    """Select the best record from a group of duplicates."""
    if len(group) == 1:
        return group
    
    # Sort by Age (descending), then by Work_Experience (descending)
    # Handle NaN values by treating them as lowest priority
    group_sorted = group.sort_values(
        ['Age', 'Work_Experience'], 
        ascending=[False, False],
        na_position='last'
    )
    
    return group_sorted.iloc[[0]]  # Return first row as DataFrame

# Apply deduplication
df_deduplicated = df_dedup.groupby('Customer ID').apply(select_best_record).reset_index(drop=True)

print(f"After deduplication: {len(df_deduplicated)} records")
print(f"Records removed: {len(df) - len(df_deduplicated)}")
print(f"Unique Customer IDs: {df_deduplicated['Customer ID'].nunique()}")



DEDUPLICATION PROCESS
After deduplication: 8363 records
Records removed: 2332
Unique Customer IDs: 8363


/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_10523/462901987.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_deduplicated = df_dedup.groupby('Customer ID').apply(select_best_record).reset_index(drop=True)


In [8]:

# Show some examples of deduplication decisions
print(f"\nExamples of deduplication decisions:")
print(f"{'Customer ID':<15} {'Original Records':<18} {'Selected Record Criteria'}")
print("-" * 80)

for customer_id in list(duplicate_ids)[:5]:  # Show first 5 examples
    original_records = df[df['Customer ID'] == customer_id].copy()
    selected_record = df_deduplicated[df_deduplicated['Customer ID'] == customer_id].iloc[0]
    
    # Show the decision criteria
    ages = original_records['Age'].tolist()
    work_exps = original_records['Work_Experience'].tolist()
    selected_age = selected_record['Age']
    selected_work_exp = selected_record['Work_Experience']
    
    decision_reason = f"Age: {selected_age} (from {ages}), Work_Exp: {selected_work_exp} (from {work_exps})"
    print(f"{customer_id:<15} {len(original_records):<18} {decision_reason}")

# Summary statistics
print(f"\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)
print(f"Original records: {len(df)}")
print(f"Deduplicated records: {len(df_deduplicated)}")
print(f"Records removed: {len(df) - len(df_deduplicated)}")
print(f"Duplicate Customer IDs processed: {len(duplicate_ids)}")
print(f"Most frequently changing column: {max(column_changes.items(), key=lambda x: x[1])[0] if column_changes else 'None'}")

print( {
    'column_changes': dict(column_changes),
    'duplicate_ids_count': len(duplicate_ids),
    'records_removed': len(df) - len(df_deduplicated),
    'changing_columns_per_id': changing_columns_per_id
})


Examples of deduplication decisions:
Customer ID     Original Records   Selected Record Criteria
--------------------------------------------------------------------------------
460394          2                  Age: 32 (from [32, 32]), Work_Exp: 9.0 (from [9.0, 9.0])
460962          2                  Age: 61 (from [55, 61]), Work_Exp: 9.0 (from [9.0, 9.0])
460333          2                  Age: 42 (from [42, 40]), Work_Exp: 10.0 (from [10.0, 12.0])
467208          2                  Age: 41 (from [41, 39]), Work_Exp: nan (from [nan, nan])
466301          2                  Age: 52 (from [50, 52]), Work_Exp: 1.0 (from [0.0, 1.0])

SUMMARY STATISTICS
Original records: 10695
Deduplicated records: 8363
Records removed: 2332
Duplicate Customer IDs processed: 2332
Most frequently changing column: Age
{'column_changes': {'Age': 4118, 'Work_Experience': 2284}, 'duplicate_ids_count': 2332, 'records_removed': 2332, 'changing_columns_per_id': [{'Customer ID': np.int64(460394), 'num_records':

In [9]:
df_deduplicated[df_deduplicated['Customer ID'].isin(duplicate_ids)].isna().sum()

Customer ID          0
Gender               0
Ever_Married        46
Age                  0
Graduated           22
Profession          34
Work_Experience    241
Spending_Score       0
Family_Size        104
dtype: int64

In [10]:
df[df['Customer ID'].isin(duplicate_ids)].isna().sum()

Customer ID          0
Gender               0
Ever_Married        92
Age                  0
Graduated           44
Profession          68
Work_Experience    482
Spending_Score       0
Family_Size        208
dtype: int64

In [11]:
df_deduplicated.to_csv("../data/Customer Segmentation DeDup.csv", index=False)